In [0]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image
import cv2
import pandas


In [0]:
import sys 
from termcolor import colored, cprint 

def fullShadeWidget(shadedPics, emotion1, emotion2, emotion3):
    """Iterates through the photos in shadedPics and displays them with
    an interactive widget. A legend for the different emotions is also
    produced.
    @shadedPics: path to folder of images to be displayed
    @emotion1: first emotion name
    @emotion2: second emotion name
    @emotion3: third emotion name
    """
    #following three functions taken from:
    #https://www.geeksforgeeks.org/print-colors-python-terminal/
    def prRed(skk): 
        print("\033[91m {}\033[00m" .format(skk))
    def prGreen(skk): 
        print("\033[92m {}\033[00m" .format(skk))
    def prBlue(skk): 
        print("\033[96m {}\033[00m" .format(skk)) 
        
    
    img_lst = []
    for i in sorted(os.listdir(shadedPics)):
      img_lst += [i]

    @widgets.interact
    def f(x=(0, len(img_lst) - 1)):
        name = img_lst[x]
        print(name)
        print()
        prRed("RED")
        print(" = " + emotion1)
        print()
        prGreen("GREEN")
        print(" = " + emotion2)
        print()
        prBlue("BLUE")
        print(" = " + emotion3)
        print()
        img = mpimg.imread(shadedPics + "/" + name)
        plt.imshow(img)
        # USE MATPLOT to change make bigger all the stuff you know...
        plt.show()


In [0]:

class RGBTransform(object):
    #code taken from William Chargin
    #https://gist.github.com/WChargin/d8eb0cbafc4d4479d004
    #article he wrote about code:
    #http://engineering.khanacademy.org/posts/making-thumbnails-fast.htm

    #this code allows us to shade an image
    """A description of an affine transformation to an RGB image.
    This class is immutable.
    Methods correspond to matrix left-multiplication/post-application:
    for example,
        RGBTransform().multiply_with(some_color).desaturate()
    describes a transformation where the multiplication takes place first.
    Use rgbt.applied_to(image) to return a converted copy of the given image.
    For example:
        grayish = RGBTransform.desaturate(factor=0.5).applied_to(some_image)
    """

    def __init__(self, matrix=None):
        self._matrix = matrix if matrix is not None else np.eye(4)

    def _then(self, operation):
        return RGBTransform(np.dot(_embed44(operation), self._matrix))

    def desaturate(self, factor=1.0, weights=(0.299, 0.587, 0.114)):
        """Desaturate an image by the given amount.
        A factor of 1.0 will make the image completely gray;
        a factor of 0.0 will leave the image unchanged.
        The weights represent the relative contributions of each channel.
        They should be a 1-by-3 array-like object (tuple, list, np.array).
        In most cases, their values should sum to 1.0
        (otherwise, the transformation will cause the image
        to get lighter or darker).
        """
        weights = _to_rgb(weights, "weights")

        # tile: [wr, wg, wb]  ==>  [[wr, wg, wb], [wr, wg, wb], [wr, wg, wb]]
        desaturated_component = factor * np.tile(weights, (3, 1))
        saturated_component = (1 - factor) * np.eye(3)
        operation = desaturated_component + saturated_component

        return self._then(operation)

    def multiply_with(self, base_color, factor=1.0):
        """Multiply an image by a constant base color.
        The base color should be a 1-by-3 array-like object
        representing an RGB color in [0, 255]^3 space.
        For example, to multiply with orange,
        the transformation
            RGBTransform().multiply_with((255, 127, 0))
        might be used.
        The factor controls the strength of the multiplication.
        A factor of 1.0 represents straight multiplication;
        other values will be linearly interpolated between
        the identity (0.0) and the straight multiplication (1.0).
        """
        component_vector = _to_rgb(base_color, "base_color") / 255.0
        new_component = factor * np.diag(component_vector)
        old_component = (1 - factor) * np.eye(3)
        operation = new_component + old_component

        return self._then(operation)

    def mix_with(self, base_color, factor=1.0):
        """Mix an image by a constant base color.
        The base color should be a 1-by-3 array-like object
        representing an RGB color in [0, 255]^3 space.
        For example, to mix with orange,
        the transformation
            RGBTransform().mix_with((255, 127, 0))
        might be used.
        The factor controls the strength of the color to be added.
        If the factor is 1.0, all pixels will be exactly the new color;
        if it is 0.0, the pixels will be unchanged.
        """
        base_color = _to_rgb(base_color, "base_color")
        operation = _embed44((1 - factor) * np.eye(3))
        operation[:3, 3] = factor * base_color

        return self._then(operation)

    def get_matrix(self):
        """Get the underlying 3-by-4 matrix for this affine transform."""
        return self._matrix[:3, :]

    def applied_to(self, image):
        """Apply this transformation to a copy of the given RGB* image.
        The image should be a PIL image with at least three channels.
        Specifically, the RGB and RGBA modes are both supported, but L is not.
        Any channels past the first three will pass through unchanged.
        The original image will not be modified;
        a new image of the same mode and dimensions will be returned.
        """

        # PIL.Image.convert wants the matrix as a flattened 12-tuple.
        # (The docs claim that they want a 16-tuple, but this is wrong;
        # cf. _imaging.c:767 in the PIL 1.1.7 source.)
        matrix = tuple(self.get_matrix().flatten())

        channel_names = image.getbands()
        channel_count = len(channel_names)
        if channel_count < 3:
            raise ValueError("Image must have at least three channels!")
        elif channel_count == 3:
            return image.convert('RGB', matrix)
        else:
            # Probably an RGBA image.
            # Operate on the first three channels (assuming RGB),
            # and tack any others back on at the end.
            channels = list(image.split())
            rgb = PIL.Image.merge('RGB', channels[:3])
            transformed = rgb.convert('RGB', matrix)
            new_channels = transformed.split()
            channels[:3] = new_channels
            return PIL.Image.merge(''.join(channel_names), channels)

    def applied_to_pixel(self, color):
        """Apply this transformation to a single RGB* pixel.
        In general, you want to apply a transformation to an entire image.
        But in the special case where you know that the image is all one color,
        you can save cycles by just applying the transformation to that color
        and then constructing an image of the desired size.
        For example, in the result of the following code,
        image1 and image2 should be identical:
            rgbt = create_some_rgb_tranform()
            white = (255, 255, 255)
            size = (100, 100)
            image1 = rgbt.applied_to(PIL.Image.new("RGB", size, white))
            image2 = PIL.Image.new("RGB", size, rgbt.applied_to_pixel(white))
        The construction of image2 will be faster for two reasons:
        first, only one PIL image is created; and
        second, the transformation is only applied once.
        The input must have at least three channels;
        the first three channels will be interpreted as RGB,
        and any other channels will pass through unchanged.
        To match the behavior of PIL,
        the values of the resulting pixel will be rounded (not truncated!)
        to the nearest whole number.
        """
        color = tuple(color)
        channel_count = len(color)
        extra_channels = tuple()
        if channel_count < 3:
            raise ValueError("Pixel must have at least three channels!")
        elif channel_count > 3:
            color, extra_channels = color[:3], color[3:]

        color_vector = np.array(color + (1, )).reshape(4, 1)
        result_vector = np.dot(self._matrix, color_vector)
        result = result_vector.flatten()[:3]

        full_result = tuple(result) + extra_channels
        rounded = tuple(int(round(x)) for x in full_result)

        return rounded


def _embed44(matrix):
    """Embed a 4-by-4 or smaller matrix in the upper-left of I_4."""
    result = np.eye(4)
    r, c = matrix.shape
    result[:r, :c] = matrix
    return result


def _to_rgb(thing, name="input"):
    """Convert an array-like object to a 1-by-3 numpy array, or fail."""
    thing = np.array(thing)
    assert thing.shape == (3, ), (
        "Expected %r to be a length-3 array-like object, but found shape %s" %
            (name, thing.shape))
    return thing


In [0]:
def shadeFace(faceLocations, imagePath, output, firstVal, secondVal, thirdVal, name):
    """Shades the faces in one image.
    @facelocations: a 2D array representing the locations of all the faces in an image. Each
    inner array contains 4 ints representing the pixels of the face's 4 corners. The 4 corners
    must be left, upper, right, lower.
    @imagePath: a path to the image to be shaded
    @output: a path to the output folder where the new image 
    @firstVal: an array of the proportions of the first emotion
    @secondVal: an array of the proportions of the second emotion
    @thirdVal: an array of the proportions of the third emotion
    @name: the name for the new shaded image
    """
    #read image
    img = Image.open(imagePath)
    
    #store the cropped faces into a list
    imgFaces = []
    for i in faceLocations:
        if (len(i) != 0):
            imgFaces.append(img.crop((i[0], i[1], i[2], i[3])))
        else:
            imgFaces.append(0)
    
    #shade each cropped face accordng to firstVal, secondVal, and thirdVal and store in newFaces
    newFaces = []
    count = 0
    for i in imgFaces:
        if (i != 0):
            shadedFace = i.convert('RGB')
            redMix = firstVal[count] * 255
            greenMix = secondVal[count] * 255
            blueMix = thirdVal[count] * 255
            shadedFace = RGBTransform().mix_with((redMix, greenMix, blueMix),factor=.6).applied_to(shadedFace)
            newFaces.append(shadedFace)
            count += 1
        else:
            count += 1
        
    #paste each new shaded face onto a copy of the original image
    count = 0
    img = img.convert('RGB')
    for i in faceLocations:
        #Image.Image.paste(img, newFaces[count], (i[0], i[1], i[2], i[3]) )
        if (len(i) != 0):
            img.paste(newFaces[count], (i[0], i[1], i[2], i[3]))
            count += 1
    
    
    #sets working directory to 'output'
    os.chdir(output)
    #saves pic with shaded faces into 'output'
    img.save(name + ".jpg", "JPEG")

In [0]:
def aggregateShadeFace(imagePath, DFPath, output, faceLocations):
    """Iterates through each photo in an folder and shades its individual faces according
    to the emotions of each person.
    @imagePath: path to the folder with the unshaded images
    @DFPath: path to the folder with each individual image's dataframe. Dataframes must have a
    column of the proportions of emotions for each face in the image and nothing else. Dataframes
    must be in the same order in the folder as the images in imagePath.
    @output: path to folder where shaded images will be saved
    @faceLocations: a 3D array representing all the facelocations for all the faces for all
    the images. 1st dimension should be the image, 2nd dimension should be the face, 3d dimension should
    be the face corners. The corners are pixel ints and must be saved in ascending order.  
    """
    #set the working directory to DFPath and store dataframe names to DFNames
    os.chdir(DFPath)
    DFNames = os.listdir()
    #set the working directory to imagePath and store image names to imageNames
    os.chdir(imagePath)
    imageNames = os.listdir()
    #determine if using 2 or 3 emotions
    testDFPath = DFPath + "/" + DFNames[0]
    testDF = pandas.read_csv(testDFPath)
    hasThreeEmotions = True
    if (len(testDF.columns) != 3):
        hasThreeEmotions = False
    
    #iteratire through each image. Call shadeFace for that image's faceLocations and emotions
    for i in np.arange(len(imageNames)):
        if (hasThreeEmotions):
            theDF = DFPath + "/" + DFNames[i]
            theDF = pandas.read_csv(theDF)
            theFile = imagePath + "/" + imageNames[i]
            firstVal = theDF.iloc[:,0]
            secondVal = theDF.iloc[:,1]
            thirdVal = theDF.iloc[:,2]
            firstVal = pandas.Series.to_numpy(firstVal)
            secondVal = pandas.Series.to_numpy(secondVal)
            thirdVal = pandas.Series.to_numpy(thirdVal)
            shadeFace(faceLocations[i], theFile, output, firstVal, secondVal, thirdVal, "faceshaded_" + imageNames[i])
        if (not hasThreeEmotions):
            theDF = DFPath + "/" + DFNames[i]
            theDF = pandas.read_csv(theDF)
            theFile = imagePath + "/" + imageNames[i]
            firstVal = theDF.iloc[:,0]
            secondVal = theDF.iloc[:,1]
            firstVal = pandas.Series.to_numpy(firstVal)
            secondVal = pandas.Series.to_numpy(secondVal)
            thirdVal = np.zeros(len(firstVal))
            shadeFace(faceLocations[i], theFile, output, firstVal, secondVal, thirdVal, "faceshaded_" + imageNames[i])
            

In [0]:
def imageShading (imagePath, output, firstVal, secondVal, thirdVal, name):
    '''Shades an entire image from ImagePath based on values. Saves as name to output.
    @image_path: path to image that will be shaded
    @output: where shaded image will be stored
    @firstVal: decimal representing strength of first emotion
    @secondVal: decimal representing strength of second emotion
    @thirdVal: decimal representing strength of third emotion
    @name: what the shaded image will be named '''
    #code does not work if more than 3 emotions; does work with 2 emotions
    #if you have only 3 emotions, set thirdVal to 0
    
    
    #inspired by this post:
    #https://stackoverflow.com/questions/32578346/how-to-change-color-of-image-using-python
    #This code uses a python library (transofmrs.py) developed by William Chargin
    #source: https://gist.github.com/WChargin/d8eb0cbafc4d4479d004
    #Chargin's article about the code:
    #http://engineering.khanacademy.org/posts/making-thumbnails-fast.htm
    
    
    #firstVal is red, secondVal is green, thirdVal is blue
    
    pic = Image.open(imagePath)
    pic = pic.convert('RGB')
    
    #determine what proportion of RGB the image should be
    redMix = firstVal * 255
    greenMix = secondVal * 255
    blueMix = thirdVal * 255
    
    #shade the pic according to proportions, with a strength of 0.3
    pic = RGBTransform().mix_with((redMix, greenMix, blueMix),factor=.75).applied_to(pic)
    
    #sets working directory to 'output'
    os.chdir(output)
    #saves pic into 'output'
    pic.save(name + ".jpg", "JPEG")

In [0]:
def aggregateFullShade(df, input, output):
    """Iterates through all the photos in a folder and shades the entire image according to their emotions.
    @df: dataframe containing the proportion of emotions for each image. Can have 2 or 3 emotions. Dataframe
    must have the columns in the order "picture names, emotions, dominant emotion"
    @input: folder with images to be shaded
    @output: folder where shaded images will be saved
    """
    #check to see new dataframe format
    #chop off first and last column of df
    df.iloc[:,-1]
    newDF = df
    newDF.drop([0])
    #list of emotion names
    emotions = newDF.columns
    #set working directory to input and get list of file names
    os.chdir(input)
    photoNames = os.listdir()
    #iterate through and shade each file
    for i in np.arange(len(photoNames)):
        if (len(emotions) == 3):
            theFile = input + "/" + photoNames[i]
            imageShading(theFile, output, newDF.loc[i][1], newDF.loc[i][2], newDF.loc[i][3], "shaded_" + photoNames[i])
        else:
            theFile = input + "/" + photoNames[i]
            imageShading(theFile, output, newDF.loc[i][1], newDF.loc[i][2], 0, "shaded_" + photoNames[i])

In [0]:
#example of how functions can be called to create a widget displaying the different shaded faces

theFaceLocations = [[[253, 425, 408, 580], [811, 440, 1034, 663], [306, 175, 414, 283]],
 [[811, 440, 1034, 663], [242, 407, 428, 593], [318, 187, 426, 295]],
 [[253, 425, 408, 580], [811, 440, 1034, 663], [311, 168, 440, 297]],
 [[811, 440, 1034, 663], [270, 425, 425, 580], [306, 163, 414, 271]],
 [[270, 425, 425, 580], [795, 409, 1062, 676]],
 [[811, 415, 1034, 638], [270, 425, 425, 580]],
 [[835, 415, 1058, 638]],
 [[885, 489, 1108, 712]],
 [[885, 464, 1108, 687]],
 [[210, 383, 339, 512], [854, 468, 1122, 720]],
 [[825, 468, 1092, 720], [882, 57, 1067, 242], [159, 407, 345, 593]],
 [[825, 52, 1092, 320], [180, 387, 366, 573], [812, 420, 1133, 720]],
 [[825, 439, 1092, 706], [201, 391, 356, 546]],
 [[201, 366, 386, 552], [825, 439, 1092, 706], [835, 68, 1058, 291]],
 [[825, 379, 1092, 647], [218, 391, 373, 546]]]

#example paths
#contains paths to original, unshaded images
newInput = r"C:\Users\Jason\Matt School Stuff\finalInput"
#where the images will be saved once shaded
newOutput = r"C:\Users\Jason\Matt School Stuff\finalTest"
#path to DFs for each image's faces
newDFPath = r"C:\Users\Jason\Matt School Stuff\finalDFs"

aggregateShadeFace(newInput, newDFPath, newOutput, theFaceLocations)

fullShadeWidget(newOutput, "disgust", "surprise", "happiness")

interactive(children=(IntSlider(value=7, description='x', max=14), Output()), _dom_classes=('widget-interact',…